# Tutorial3: estimate 3D emission from image-plane directly

---
This tutorial demonstrates the recovery of 3D emission from a gravitationally lensed image plane. The unknown emission is modeled using a coordinate-based neural network (NeRF).

In [1]:
import bhnerf
from bhnerf.optimization import LogFn
import numpy as np
import os
from datetime import datetime
from astropy import units

# Runing on 2 GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ils1v478 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-12-21 16:48:51.111573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [2]:
"""
Generate gravitationally lensed image-plane of a hot-spot
"""
spin = 0.2
inclination = np.deg2rad(60.0)      
nt = 64                             # number of image frames
fov_M = 16.0                        # field of view (M)
rmax = fov_M / 2
t_frames = np.linspace(0, 1.0, nt) * units.hr

# Compute geodesics (see Tutorial1)
geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=64, num_beta=64, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2]
)
t_injection = -float(geos.r_o)
rmin = float(geos.r.min())

# Keplerian prograde velocity field
Omega = np.sign(spin + np.finfo(float).eps) * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))

emission_0 = bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=0.0,
    orbit_radius=5.5,
    std=0.7,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2')
)
image_plane = bhnerf.emission.image_plane_dynamics(emission_0, geos, Omega, t_frames, t_injection)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [3]:
"""
Optimize network paremters to recover the 3D emission (as a continuous function) from observations 
Note that logging is done using tensorboardX. To view the tensorboard (from the main directory):
    `tensorboard --logdir runs`
"""
batchsize = 6
z_width = 4      # maximum disk width [M]
hparams = {'num_iters': 5000, 'lr_init': 1e-4, 'lr_final': 1e-6}

# Logging 
current_time = datetime.now().strftime('%Y-%m-%d.%H:%M:%S')
runname = 'tutorial3/recovery.{}'.format(current_time)
writer = bhnerf.optimization.SummaryWriter(logdir='../runs/{}'.format(runname))
writer.add_images('emission/true', bhnerf.utils.intensity_to_nchw(emission_0), global_step=0)
log_fns = [
    LogFn(lambda opt: writer.add_scalar('log_loss/train', np.log10(np.mean(opt.loss)), global_step=opt.step)), 
    LogFn(lambda opt: writer.recovery_3d(rmin, rmax, z_width, emission_true=emission_0)(opt), log_period=200)
]

# Optimization
train_step = bhnerf.optimization.TrainStep.image(t_frames, image_plane)
predictor = bhnerf.network.NeRF_Predictor()
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], rmax, z_width)
optimizer = bhnerf.optimization.Optimizer(hparams, predictor, raytracing_args, checkpoint_dir='../checkpoints/{}'.format(runname))
optimizer.run(batchsize, train_step, raytracing_args, log_fns=log_fns)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


iteration:   0%|          | 0/5000 [00:00<?, ?it/s]

In [4]:
"""
Visualize the recovered 3D emission
This visualization requires ipyvolume: https://ipyvolume.readthedocs.io/en/latest/
"""
emission_estimate = bhnerf.network.sample_3d_grid(predictor.apply, optimizer.state.params, rmin, rmax, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_estimate, fov=fov_M, level=[0, 0.2, 0.7])